In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import os
import pickle
import json
import re

In [2]:
tf.test.is_gpu_available()

True

In [3]:
MAX_SEQ_LEN = 100 + 

# load pretrained roberta
import transformers
from tensorflow.keras import layers

class ROBERTA(transformers.TFRobertaModel):

    def __init__(self, config, *inputs, **kwargs):
        super(ROBERTA, self).__init__(config, *inputs, **kwargs)
        self.roberta.call = tf.function(self.roberta.call)

input_ids = layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32)
roberta = ROBERTA.from_pretrained('roberta-base')
roberta_encodings = roberta([input_ids])[0]
doc_encoding = tf.squeeze(roberta_encodings[:, 0:1, :], axis=1) # Keep [CLS] token encoding
doc_encoding = layers.Dropout(0.1)(doc_encoding) # Apply dropout
outputs = layers.Dense(1, activation='sigmoid', name='outputs')(doc_encoding)
model = tf.keras.models.Model(inputs=[input_ids], outputs=[outputs])

# compile
optimizer = tf.keras.optimizers.Adam(lr=5e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "roberta"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  124645632 
Total params: 124,645,632
Trainable params: 124,645,632
Non-trainable params: 0
_________________________________________________________________
None
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
roberta (ROBERTA)            ((None, 50, 768), (None,  124645632 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1, 768)]          0         
_________________________________________________________________
tf_op_layer_Squeeze (TensorF [(None, 768)]             0         
_________________________

In [24]:
embeddings = roberta.get_input_embeddings()
print(type(embeddings))

<class 'transformers.modeling_tf_roberta.TFRobertaEmbeddings'>


In [27]:
embed = embeddings.position_embeddings
embed(np.array([1, 2, 3]))

<tf.Tensor: id=11474, shape=(3, 768), dtype=float32, numpy=
array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.03460693, -0.01693726, -0.08953857, ..., -0.05422974,
         0.02912903,  0.01733398],
       [ 0.04006958,  0.00033379,  0.05801392, ...,  0.01217651,
        -0.01846313,  0.07354736]], dtype=float32)>